### to include in the other file before running this one

In [ ]:
# # import settings and functions
# %run ./../../../imports.ipynb

# # from ana dense
# prefix = "c5_brick_NL_dense_ref_"
# csv_name = 'dataset_loop_complete_scaled.csv'

# run_analysis = True

# color_lims = True

# exe = hdiv_data_driven_diffusion_snes
# sumanalys = "sumanalys.csv"
# ana_name = "ana_DD_brick"

if 'additions' not in locals():
    additions = ""

#### Get dataset and scaling

In [ ]:
!cp ./../../../datasets_brick/scaling.in .
!cp ./../../../datasets_brick/NL_dataset_creation/{csv_name} .
!cp ./../../../datasets_brick/regular/{csv_name} .

!ls ./../../../datasets_brick/scaling.in
!ls ./../../../datasets_brick/NL_dataset_creation/{csv_name}
!ls ./../../../datasets_brick/regular/{csv_name}


scal = np.genfromtxt('./scaling.in')
scaling = scal
print(scaling)

### Settings for refinement

In [ ]:
# if variable 'ref' does not exist, create it
if 'ref' not in locals():
    ref = ""

refinement_style = 0
ref_iter_num = 0
refinement_control = ""

if ref:
    print("refinement: ")

    # refinement style: 
    # 0 - no refinement
    # 1 - p refinement (order)
    # 2 - h refinement (mesh)
    # 3 - hp refinement
    refinement_style = 3

    # number of refinements
    ref_iter_num = 4

    # refine mesh due to error indicators only on boundary
    boundary_ony = "-refine_h_boundary_only"

    # order refinement control
    # > ord_ref_control * average error -> refine
    ord_ref_control = 3.0
    # mesh refinement control
    # > mesh_ref_control * average error -> refine
    mesh_ref_control = 4.0

    # DD refinement control
    # d > tol_distance_ref * d_ave -> do not refine
    tol_distance_ref = 1.0
    # d > tol_distance_var_ref * d_var 
    # &&
    # d_var > tol_distance_var_ave * d_ave
    #  -> refine
    tol_distance_var_ref = 2.0
    tol_distance_var_ave = 0.5

    refinement_control = f"-ref_control {ord_ref_control} -ref_control_mesh {mesh_ref_control} -tol_distance_ref {tol_distance_ref} -tol_distance_var_ref {tol_distance_var_ref} -tol_distance_var_ave {tol_distance_var_ave} {boundary_ony}"

# print refinement style and number
print(f"refinement style: {refinement_style}")
print(f"number of refinements: {ref_iter_num}")

print(refinement_control)


### Settings for monte

In [1]:
monte_control = ""
monte_num = 30

if 'monte' not in locals():
    monte = ""

if monte:
    monte_control = f"-monte_carlo {monte_num}"
    
print(monte_control)

### General settings

In [ ]:
# approximation order for temperatures
params.order = 3
# number of processors
params.nproc = 1 
# calculation of jumps only works with 1 processor
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"

# naming for the csv file
naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

In [ ]:
# which file?
params.list_mesh_files = []
params.element_sizes = []

# 0
params.list_mesh_files.append("./../brick_20.cub")
params.element_sizes.append(5)
# 1
params.list_mesh_files.append("./../brick_15.cub")
params.element_sizes.append(4)
# 2
params.list_mesh_files.append("./../brick_10.cub")
params.element_sizes.append(3)
# 3
params.list_mesh_files.append("./../brick_7.cub")
params.element_sizes.append(2)
# 4
params.list_mesh_files.append("./../brick_6.cub")
params.element_sizes.append(1)
# 5
params.list_mesh_files.append("./../brick_4.cub")
params.element_sizes.append(1)
# 6
params.list_mesh_files.append("./../brick_q.cub")
params.element_sizes.append(10)

h = params.element_sizes
cub_names = params.list_mesh_files
input_names = []
for i in cub_names:
    input_names.append(i[:-3]+"h5m")
count = len(h)
print(cub_names)
print(input_names)

# which mesh to use
input_number = 6

# # which mesh to use
# input_number = 3

input_mesh = params.list_mesh_files[input_number]
input_name = input_names[input_number]
input_cub = cub_names[input_number]
print(input_mesh)

In [ ]:
params.csv_tree_file = csv_name

In [ ]:
!pwd
print("1")

In [ ]:
# # start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()
print("2")

### Run analysis

In [ ]:
if run_analysis:
    !rm out*[0-9]*
    # params.part_file = input_names[input_number]

    print("hello")

    # params.mesh_file = cub_names[0]
    # params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {input_cub} -nparts {params.nproc} -output_file {input_name} -dim 2 -adj_dim 1
    !mpirun -np {params.nproc} {exe} -file_name {input_name} -my_order {params.order} {jumps} -csv_tree_file {params.csv_tree_file} -scaling 1 -snes_max_it 500 -print_integ -refinement_style {refinement_style} -ref_iter_num {ref_iter_num} {refinement_control} {additions} {monte_control}
    #  -use_line -data_dim 4 -my_dummy_k 100.0

### Post-process

In [ ]:
name_vtk_integ = "out_integ_pts_"+csv_name[:-4]
name_vtk_result = "out_result_"+csv_name[:-4]
name_vtk_error = "out_error_"+csv_name[:-4]
name_vtk_moab = "out_moab_"+csv_name[:-4]

# monte
name_vtk_monte_errors = "out_monte_errors_"+csv_name[:-4]
name_vtk_monte_integ = "out_monte_integ_"+csv_name[:-4]
name_vtk_monte_sigma = "out_monte_sigma_"+csv_name[:-4]
name_vtk_monte_integ_mean_var = "out_integ_pts_mean_var_"+csv_name[:-4]


In [ ]:
if run_analysis:    
    out_to_vtk = !ls -c1 out_integ_pts_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_integ}.vtk

    out_to_vtk = !ls -c1 out_result*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_result}.vtk

    out_to_vtk = !ls -c1 out_error*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_error}.vtk
    # !mbconvert out_moab_5.h5m {name_vtk_moab}.vtk
    # !rm out_*h5m

    out_to_vtk = !ls -c1 out_monte_ele_errors_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_monte_errors}.vtk

    out_to_vtk = !ls -c1 out_monte_integ_star_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_monte_integ}.vtk

    out_to_vtk = !ls -c1 out_sigma_ele_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_monte_sigma}.vtk

    out_to_vtk = !ls -c1 out_integ_pts_mean_var_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_monte_integ_mean_var}.vtk




In [ ]:
# import settings and functions
%run ./../plot_for_hdiv_errors.ipynb

In [ ]:
# import settings and functions
%run ./../plot_for_DD.ipynb

In [ ]:
if monte:
    %run ./../plot_for_monte.ipynb